In [25]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import scipy as sp
import quandl
import functools
import seaborn as sns
import plotnine as p9
from datetime import datetime

# 0. Setup

In [131]:
# save private key to: ../srv/mykey/.quandlapikey
# loading private key
quandl.read_key("../srv/mykey/.quandlapikey")
MonthSymbolList = ["H", "M", "U", "Z"]
start_date = "2020-12-3"
end_date = "2022-08-31"
OptionCodeList = [m+y for y in ("2020", "2021", "2022") for m in MonthSymbolList][4:-2]
OptionCodeList

['H2021', 'M2021', 'U2021', 'Z2021', 'H2022', 'M2022']

In [44]:
try:
    # Do not recreate if it already exists
    fetch_quandl
    print("fetch_quandl already defined")
except NameError:
    @functools.lru_cache(maxsize=16)
    def fetch_quandl(my_sec):
        print("Fetching from Quandl")
        qdata = quandl.get(my_sec)
        return qdata
    print("fetch_quandl has now been newly defined")

fetch_quandl already defined


In [139]:
def get_futures_data(name, option_code_list, s=start_date, e=end_date):
    Future = pd.DataFrame()
    Dtt = pd.DataFrame()
    f_ls, dtt_ls = [], []
    print(name)
    for code in option_code_list:
        sec_name = "OWF/%s_%s_IVM" % (name, code)
        print("Getting data: " + sec_name)
        data = quandl.get(sec_name, start_date=s, 
                      end_date=e)
        f_ls.append(data.Future)
        dtt_ls.append(data.DtT)
    Future = pd.concat(f_ls, axis=1, join="outer")
    Dtt = pd.concat(dtt_ls, axis=1, join="outer")
    Future.columns = option_code_list
    Dtt.columns = option_code_list
    return Future, Dtt

In [132]:
def cal_sec_mon_future(name, option_code_list, s, e, boundary=30.0):
    future, dtt =  get_futures_data(name, option_code_list, s, e)
    df = dtt[dtt > boundary]
    ls = df.idxmin(axis=1)
    future_price = pd.DataFrame(columns=[name])
    for idx, col in ls.items():
        try:
            future_price.loc[idx, name] = future.loc[idx, col]
        except KeyError:
            continue
    return future_price

# 1. Data

In [140]:
# Student ID: 12375213
# 1. ICE_B_B versus ICE_G_G×0.1147
# 3. CBT_FV_FV versus CBT_TU_TU × 1.1066
# W, X; Y, Z
# X-W; Z-Y
ICE_B_B = cal_sec_mon_future("ICE_B_B", OptionCodeList, s=start_date, e=end_date)
ICE_G_G = cal_sec_mon_future("ICE_G_G", OptionCodeList, s=start_date, e=end_date)
ICE_B_B_ICE_G_G = ICE_B_B.merge(ICE_G_G, left_index=True, right_index=True)

CBT_FV_FV = cal_sec_mon_future("CBT_FV_FV", OptionCodeList, s=start_date, e=end_date)
CBT_TU_TU = cal_sec_mon_future("CBT_TU_TU", OptionCodeList, s=start_date, e=end_date)
CBT_FV_FV_CBT_TU_TU = CBT_FV_FV.merge(CBT_TU_TU, left_index=True, right_index=True)

ICE_B_B
Getting data: OWF/ICE_B_B_H2021_IVM
Getting data: OWF/ICE_B_B_M2021_IVM
Getting data: OWF/ICE_B_B_U2021_IVM
Getting data: OWF/ICE_B_B_Z2021_IVM
Getting data: OWF/ICE_B_B_H2022_IVM
Getting data: OWF/ICE_B_B_M2022_IVM
ICE_G_G
Getting data: OWF/ICE_G_G_H2021_IVM
Getting data: OWF/ICE_G_G_M2021_IVM
Getting data: OWF/ICE_G_G_U2021_IVM
Getting data: OWF/ICE_G_G_Z2021_IVM
Getting data: OWF/ICE_G_G_H2022_IVM
Getting data: OWF/ICE_G_G_M2022_IVM
CBT_FV_FV
Getting data: OWF/CBT_FV_FV_H2021_IVM
Getting data: OWF/CBT_FV_FV_M2021_IVM
Getting data: OWF/CBT_FV_FV_U2021_IVM
Getting data: OWF/CBT_FV_FV_Z2021_IVM
Getting data: OWF/CBT_FV_FV_H2022_IVM
Getting data: OWF/CBT_FV_FV_M2022_IVM
CBT_TU_TU
Getting data: OWF/CBT_TU_TU_H2021_IVM
Getting data: OWF/CBT_TU_TU_M2021_IVM
Getting data: OWF/CBT_TU_TU_U2021_IVM
Getting data: OWF/CBT_TU_TU_Z2021_IVM
Getting data: OWF/CBT_TU_TU_H2022_IVM
Getting data: OWF/CBT_TU_TU_M2022_IVM


In [145]:
ICE_B_B_ICE_G_G.tail()

,ICE_B_B,ICE_G_G
2022-03-23,117.75,1054.25
2022-03-24,115.3,1045.25
2022-03-25,117.37,1047.5
2022-03-28,109.49,999.25
2022-03-29,107.71,962.75


In [142]:
CBT_FV_FV_CBT_TU_TU.tail()

,CBT_FV_FV,CBT_TU_TU
2022-05-13,112.960938,105.68125
2022-05-16,113.28125,105.742188
2022-05-17,112.671875,105.51875
2022-05-18,112.914062,105.585938
2022-05-19,113.140625,105.696875


In [146]:
S1 = ICE_B_B_ICE_G_G['ICE_G_G'] * 0.1147 - ICE_B_B_ICE_G_G['ICE_B_B']
S1 = S1.to_frame("ICE_B_B - ICE_G_G")
S1.to_excel("../data/hw1/S1.xlsx")

In [147]:
S2 = CBT_FV_FV_CBT_TU_TU["CBT_TU_TU"] * 1.1066 - CBT_FV_FV_CBT_TU_TU["CBT_FV_FV"]
S2 = S2.to_frame("CBT_TU_TU - CBT_FV_FV")
S2.to_excel("../data/hw1/S2.xlsx")

# 2. Analysis